In [12]:
# creatinga DNN with five hidden layers of 100 neuron each

import tensorflow as tf
import numpy as np

def reset_graph():
    tf.reset_default_graph()
    tf.set_random_seed(42)
    np.random.seed(42)
    

In [5]:
# reset_graph()

# n_inputs =28*28
# n_hidden1 = 300
# n_hidden2 = 150
# n_hidden3 = 50
# n_outputs = 10

# X = tf.placeholder(tf.float32,shape=(None,n_inputs),name="X")
# y = tf.placeholder(tf.int32, shape=(None),name="y")

# def elu(z, alpha=1):
#     return np.where(z<0, alpha * (np.exp(z) - 1), z)

# with tf.name_scope("dnn"):
#     hidden1= tf.layers.dense(X, n_hidden1,activation=tf.nn.elu,name="hidden1")
#     hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.elu, name="hidden2")
#     hidden3 = tf.layers.dense(hidden2, n_hidden3, activation=tf.nn.elu,name="hidden3")
#     logits = tf.layers.dense(hidden3, n_outputs, name="outputs"
#                             )

# with tf.name_scope("loss"):
#     xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
#     loss = tf.reduce_mean(xentropy, name="loss")

# learning_rate = 0.01
# with tf.name_scope("train"):
#     optimizer = tf.train.GradientDescentOptimizer(learning_rate)
#     training_op = optimizer.minimize(loss)
                   
# with tf.name_scope("eval"):
#     correct = tf.nn.in_top_k(logits,y, 1)
#     print("correct:",correct)
#     accuracy = tf.reduce_mean(tf.cast(correct,tf.float32))
    
# init = tf.global_variables_initializer()
# saver = tf.train.Saver()
# n_epochs = 40
# batch_size = 50

correct: Tensor("eval/in_top_k/InTopKV2:0", shape=(?,), dtype=bool)


In [13]:
# loading the dataset tf.examples.tutorials.mnist is depreciated

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1,28 *28)/ 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28)/ 255.0

y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

In [14]:
he_init = tf.variance_scaling_initializer()

def dnn(inputs, n_hidden_layers=5, n_neurons=100, name=None,
        activation=tf.nn.elu, initializer=he_init):
    with tf.variable_scope(name, "dnn"):
        for layer in range(n_hidden_layers):
            inputs = tf.layers.dense(inputs, n_neurons, activation=activation,
                                     kernel_initializer=initializer,
                                     name="hidden%d" % (layer + 1))
        return inputs

In [17]:
n_inputs = 28 * 28
n_outputs = 5
reset_graph()
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32,shape=(None),name="y")

dnn_outputs = dnn(X)

logits = tf.layers.dense(dnn_outputs, n_outputs, kernel_initializer=he_init, name="logits")
Y_proba = tf.nn.softmax(logits, name="Y_proba")

In [18]:
# using adam optimization for early stopping,on digits 0 to 3 and then using transfer for 5 to 0
# need softmaxwith 5 nerons
# make checkpoints and so tha we can resuse the mdoel later

learning_rate = 0.01

xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss = tf.reduce_mean(xentropy, name="loss")

optimizer = tf.train.AdamOptimizer(learning_rate)
training_op = optimizer.minimize(loss, name="training_op")

correct =tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [19]:
# creating th training set validation and teseting set
# the validation needs toimpolement early stopping

X_train1 = X_train[y_train < 5]
y_train1 = y_train[y_train < 5]
X_valid1 = X_valid[y_valid < 5]
y_valid1 = y_valid[y_valid < 5]
X_test1  = X_test[y_test < 5]
y_test1 = y_test[y_test < 5] 

In [25]:
n_epochs = 1000
batch_size =20

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

with tf.Session() as sess:
    init.run()
    
    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(X_train1))
        for rnd_indices in np.array_split(rnd_idx,len(X_train1)//batch_size):
            X_batch, y_batch = X_train1[rnd_indices], y_train1[rnd_indices]
            sess.run(training_op, feed_dict={X:X_batch, y:y_batch})
        loss_val, acc_val = sess.run([loss,accuracy], feed_dict={X:X_valid1, y:y_valid1})
        if loss_val < best_loss:
            save_path = saver.save(sess, "./my_mnist_0_to_4.ckpt")
            best_loss = loss_val
            checks_without_progress = 0
        else :
            checks_without_progress +=1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping")
                break
        print(epoch,"current loss: ", loss_val, "best loss:", best_loss, acc_val * 100)
with tf.Session() as sess:
    saver.restore(sess, "./my_mnist_0_to_4.ckpt")
    acc_test = accuracy.eval(feed_dict={X:X_test1, y:y_test1})
    print("Final accuracy:", acc_test * 100)

0 current loss:  0.17804934 best loss: 0.17804934 95.03518342971802
1 current loss:  0.119493365 best loss: 0.119493365 96.71618342399597
2 current loss:  0.09292569 best loss: 0.09292569 98.31900000572205
3 current loss:  0.26109308 best loss: 0.09292569 94.64425444602966
4 current loss:  0.44506058 best loss: 0.09292569 92.57231950759888
5 current loss:  0.20861483 best loss: 0.09292569 96.87255620956421
6 current loss:  0.10406159 best loss: 0.09292569 98.00625443458557
7 current loss:  0.17517713 best loss: 0.09292569 96.71618342399597
8 current loss:  1.63139 best loss: 0.09292569 22.009381651878357
9 current loss:  1.7801597 best loss: 0.09292569 22.009381651878357
10 current loss:  1.6627414 best loss: 0.09292569 18.725566565990448
11 current loss:  1.630582 best loss: 0.09292569 20.914776623249054
12 current loss:  1.6505957 best loss: 0.09292569 20.914776623249054
13 current loss:  1.6361288 best loss: 0.09292569 20.914776623249054
14 current loss:  1.718902 best loss: 0.09292

In [ ]:
## tuning the hyperparameters using cross validation
## __init__() method just creates the instance variables
## fit() method creates the graph, starts the session and trains
## build_graoh() to build eht egraph
## _dnn() for hidden layers
## fit () for validation set and implementing eaarly stopping
## it keeps the session open 
#  predic_proba for using trained model to predict class probabilirtis
# predictforcalling predcit proba and returing the class with highest probablility

from sklearn.base import BaseEstimator, ClassfierMixin
from sklearn.exeptions import NotFittedError

class DNNClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, n_hidden_layers=5, n_neirons=100, optimizer_class=tf.train.AdamOptimizer,
                 learning_rate = 0.01, batch_szie=20, activation=tf.nn.elu, initializer=he_init,
                 batch_norm_momentum = None,dropout_rate=None, random_state=None):
        self.n_hidden_layers = n_hidden_layers
        self.n_neurons = n_neurons
        self.optimizer_class = optimizer_class
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.activation = activation
        self.initializer = initializer
        self.batch_norm_momentum = batch_norm_momentum
        self.dropout_rate = dropout_rate
        self.random_state = random_state
        self._session = None
    
    def _dnn(self, inputs):
        # build hidden layer and batch normalisation
        if layer in range(self.n_hidden_layers):
            if self.dropout_rate:
                inputs=tf.layers.dropout(inputs, self.dropout_rate, training=self._training)
            inputs = tf.layer.dense(inputs, self.n_neurons,
                                    kernel_initializer=self.initializer,
                                    name="hidden%d" % (layer + 1))
            if (self.batch_norm_momentum):
                inputs = self.activation(inputs, name="hidden%d_out" % (layer + 1))
        return inputs
    
    def _build_graph(self, n_inputs, n_outputs):
        if self.random_state is i=not None:
            tf.set_random_seed(self.random_state)
            np.random.seed(self.random_state)
        
        X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
        y = tf.placeholder(tf.int32, shape=(None), name="y")
        
        if self.batch_norm_momrntum or self.dropour_rate:
            self._training = tf.placeholder_with_default(false, shape=(), name="training")
        else:
            self._training = None
        
        dnn_outputs = self._dnn(X)
        
        logits =tf.layers.dense(dnn_outputs, n_outputs, kernel_initializer=he_inti, name="logits")
        Y_proba = tf.nn.softmax(logits, name="Y_proba")
        
        xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
        loss = tf.reduce_mean(xentropy, name="loss")
        
        optimizer = self.optimizer_calss(learning_rate=self.learning_rate)
        training_op = optimizer.minimize(loss)
        
        correct = tf.nn.in_top_k(logits, y, 1)
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")
        
        init = tf.gloabl_variables_initializer()
        saver = tf.train.Saver()
        
        # making importatnt operations avaialble through class
        
        self._X , self._y = X, y
        self._Y_proba, self._loss = Y_proba, loss
        self._training_op, self._accuracy = training_op, accuracy
        self._init,self._saver = init, saver
        
    def close_session(self):
        if self._session:
            self.session.close()
            
    def _get_model_params(self):
        with self._graph.as_default():
            gvats = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
        return (gvat.op.name: value for gvat, value in zip(gvars,self._session.run(gvars)))
    
    def